In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Masters_Project/Datasets/OASIS3/

/content/drive/.shortcut-targets-by-id/1bqqm9-0FMOqiqyH_NIQBh-lJ36pWa7aX/Masters_Project/Datasets/OASIS3


In [22]:
!ls

 cifar-10-batches-py	  fa20		    t1w_non_ad_patients
 cifar-10-python.tar.gz   oasis-scripts    'Usage Instructions.gdoc'
 data			  t1w_ad_jpgs
 downloaded_data	  t1w_non_ad_jpgs


In [4]:
!pip install -r ../../Code/ganomaly/requirements.txt
#!pip install -r ./ganomaly/requirements.txt

  Using cached https://files.pythonhosted.org/packages/fc/d1/45be1144b03b6b1e24f9a924f23f66b4ad030d834ad31fb9e5581bd328af/numpy-1.16.4-cp37-cp37m-manylinux1_x86_64.whl
ERROR: umap-learn 0.5.1 has requirement numpy>=1.17, but you'll have numpy 1.16.4 which is incompatible.
ERROR: umap-learn 0.5.1 has requirement scikit-learn>=0.22, but you'll have scikit-learn 0.21.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.16.4 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement six~=1.15.0, but you'll have six 1.12.0 which is incompatible.
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.33.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.1.0 which is incompatible.
ERROR: mkl-fft 1.2.0 has requirement numpy==1.19.5, but you'll have numpy 1.16.4 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have mat

In [5]:
!pip install mkl-fft

  Using cached https://files.pythonhosted.org/packages/08/d6/a6aaa29fea945bc6c61d11f6e0697b325ff7446de5ffd62c2fa02f627048/numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl
ERROR: umap-learn 0.5.1 has requirement scikit-learn>=0.22, but you'll have scikit-learn 0.21.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement six~=1.15.0, but you'll have six 1.12.0 which is incompatible.
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.33.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.1.0 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.1.0 which is incompatible.
ERROR: librosa 0.8.0 has requirement joblib>=0.14, but you'll have joblib 0.13.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.15.0, bu

In [6]:
!pip install Pillow

# imports

In [18]:
import os
#import os.path
#import cv2
import glob
import torch
import random
import numpy as np
import pickle
from typing import Any, Callable, Optional, Tuple

from PIL import Image

from sklearn.model_selection import train_test_split

import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.datasets import VisionDataset
from torchvision.datasets.utils import check_integrity, download_and_extract_archive

In [9]:
print(torch.__version__)

1.2.0


In [10]:
import torchvision
torchvision.__version__

'0.4.0'

# Create Oasis3 data class

In [19]:
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])# https://github.com/yunjey/pytorch-tutorial/issues/161
        
    ]
)

In [55]:
train_image_number = 140 # Image slice from MRI scans to use for training data

base_folder = '/pyt_oasis3'
non_ad_jpg_folder = './t1w_non_ad_jpgs/' + str(train_image_number) + '/'
ad_jpg_folder = './t1w_ad_jpgs/' + str(train_image_number) + '/'

class Oasis3_train_normal(VisionDataset):
    def __init__(self,
            root: str,
            train: bool = True, # Train or test dataset
            transform: Optional[Callable] = None,
            target_transform: Optional[Callable] = None) -> None:
        super(Oasis3_train_normal, self).__init__(root, transform=transform, target_transform=target_transform)
        self.train = train # Train or test set

        # Set repeatble random number
        
        self.non_ad_data = []
        self.non_ad_targets = []

        self.ad_data = []
        self.ad_targets = []
        
        # load both datasets
        for file_path in glob.glob(ad_jpg_folder+'**.jpg'):
            with open (file_path, 'rb') as f:
                # image needs to be a PIL image
                img = Image.open(f)
                # Resize all images 176, 256, 3 -> 256, 256, 0
                dsize = (256, 256)
                resized = img.resize(dsize)
                self.ad_data.append(resized)
                self.ad_targets.extend(str(1)) # based on the jpg_folder in for loop
        self.ad_data = np.vstack(self.ad_data).reshape(-1, 256, 256)
        #self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC
        self.ad_data = self.ad_data.transpose((0, 1, 2))  # convert to HWC

        for file_path in glob.glob(non_ad_jpg_folder+'**.jpg'):
            with open (file_path, 'rb') as f:
                # image needs to be a PIL image
                img = Image.open(f)
                # Resize all images 176, 256, 3 -> 256, 256, 0
                dsize = (256, 256)
                resized = img.resize(dsize)
                self.non_ad_data.append(resized)
                self.non_ad_targets.extend(str(0)) # based on the jpg_folder in for loop
        self.non_ad_data = np.vstack(self.non_ad_data).reshape(-1, 256, 256)
        #self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC
        self.non_ad_data = self.non_ad_data.transpose((0, 1, 2))  # convert to HWC
        
        # shuffle the datasets and bucket them as test and train
        (self.non_ad_data, self.non_ad_targets) = self.shuffle_associated_arrays(self.non_ad_data, self.non_ad_targets)
        (self.ad_data, self.ad_targets) = self.shuffle_associated_arrays(self.ad_data, self.ad_targets)

        all_data = self.non_ad_data + self.ad_data
        all_targets = self.non_ad_targets + self.ad_targets
        # concat the lists and random shuffly
        random.Random(1).shuffle(all_data)
        random.Random(1).shuffle(all_targets)

        # Use scikit learn to split complete data into test and train
        self.train_data, self.test_data, self.train_targets, self.test_targets =\
         train_test_split(all_data, all_targets, test_size=0.1, random_state=42)

        # Set test or train data based on input
        if self.train:
          self.data = self.train_data
          self.targets = self.train_targets
        else:
          self.data = self.test_data
          self.targets = self.test_targets

    def shuffle_associated_arrays(self, arr1, arr2):
        c = list(zip(arr1, arr2))
        random.shuffle(c)
        arr1, arr2 = zip(*c)
        return (list(arr1), list(arr2))

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        img, target = self.data[index], self.targets[index]
        
        img = Image.fromarray(img)
        
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return (img, target)

In [56]:
# Issue is i need to make the data transformable inside __getitem__
a=Oasis3_train_normal(base_folder, False, transform=transform)

In [57]:
# Shape : (256, 256)
a.data[0].shape

(256, 256)

In [58]:
# The class should be index into which invokes the __getitem__ call
print(a[0][0])
print(a[0][1])

tensor([[[-0.9843, -0.9608, -0.9608,  ..., -0.9765, -0.9765, -1.0000],
         [-0.9686, -0.9608, -0.9843,  ..., -0.9608, -0.9686, -0.9922],
         [-0.9529, -0.9686, -1.0000,  ..., -0.9529, -0.9686, -0.9922],
         ...,
         [-0.8980, -0.9294, -0.9608,  ..., -0.9765, -0.9765, -1.0000],
         [-0.9137, -0.9373, -0.9608,  ..., -0.9686, -0.9765, -1.0000],
         [-0.9451, -0.9529, -0.9608,  ..., -0.9529, -0.9765, -1.0000]]])
0
